In [31]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN
from imblearn.metrics import classification_report_imbalanced

In [33]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [34]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [35]:
encoded_df = pd.get_dummies(df,columns=["home_ownership", "verification_status","pymnt_plan","hardship_flag","debt_settlement_flag","next_pymnt_d","application_type","issue_d","initial_list_status"])
encoded_df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,debt_settlement_flag_N,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_f,initial_list_status_w
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,1,0,1,1,0,0,0,1,0,1
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,0,1,1,0,0,0,1,0,1
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,0,1,1,0,0,0,1,0,1
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,1,0,1,1,0,0,0,1,0,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,0,1,1,0,0,0,1,0,1


In [36]:
X = encoded_df.drop(columns="loan_status")

y = encoded_df['loan_status']

In [37]:
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [39]:
X_train.shape

(51612, 95)

In [40]:
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'low_risk': 51352, 'high_risk': 51352})

In [41]:
mod = LogisticRegression(solver='lbfgs', random_state=1)
mod.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [42]:
y_pred = mod.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6459927239324608

In [43]:
confusion_matrix(y_test, y_pred)

array([[   53,    34],
       [ 5430, 11688]])

In [44]:
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.61      0.68      0.02      0.64      0.41        87
   low_risk       1.00      0.68      0.61      0.81      0.64      0.42     17118

avg / total       0.99      0.68      0.61      0.81      0.64      0.42     17205



In [45]:
X_re_smote, y_re_smote = SMOTE(random_state=1).fit_resample(
X_train, y_train
)
Counter(y_re_smote)

Counter({'low_risk': 51352, 'high_risk': 51352})

In [46]:
model_smote = LogisticRegression(solver='lbfgs', random_state=1)
model_smote.fit(X_re_smote, y_re_smote)

LogisticRegression(random_state=1)

In [47]:
y_pred_smote = model_smote.predict(X_test)
balanced_accuracy_score(y_test, y_pred_smote)

0.6546758604574334

In [48]:
confusion_matrix(y_test, y_pred_smote)

array([[   56,    31],
       [ 5723, 11395]])

In [49]:
print(classification_report_imbalanced(y_test, y_pred_smote))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.64      0.67      0.02      0.65      0.43        87
   low_risk       1.00      0.67      0.64      0.80      0.65      0.43     17118

avg / total       0.99      0.67      0.64      0.79      0.65      0.43     17205



In [50]:
cc = ClusterCentroids(random_state=1)
X_re_under, y_re_under = cc.fit_resample(X_train, y_train)
Counter(y_re_under)

Counter({'high_risk': 260, 'low_risk': 260})

In [52]:
under = LogisticRegression(solver='lbfgs', random_state=1)
under.fit(X_re_under, y_re_under)

LogisticRegression(random_state=1)

In [53]:
y_under = under.predict(X_test)
balanced_accuracy_score(y_test, y_under)

0.529127435931526

In [54]:
confusion_matrix(y_test, y_under)

array([[  53,   34],
       [9431, 7687]])

In [55]:
print(classification_report_imbalanced(y_test, y_under))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.61      0.45      0.01      0.52      0.28        87
   low_risk       1.00      0.45      0.61      0.62      0.52      0.27     17118

avg / total       0.99      0.45      0.61      0.62      0.52      0.27     17205



In [56]:
smote = SMOTEENN(random_state=0)
X_comb, y_comb = smote.fit_resample(X, y)
Counter(y_comb)


Counter({'high_risk': 68460, 'low_risk': 62011})

In [57]:
comb = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_comb, y_comb)

LogisticRegression(random_state=1)

In [58]:
y_p_c = model.predict(X_test)
balanced_accuracy_score(y_test, y_p_c)

0.6375533316412246

In [59]:
confusion_matrix(y_test, y_p_c)


array([[  61,   26],
       [7293, 9825]])

In [60]:
print(classification_report_imbalanced(y_test, y_p_c))


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.70      0.57      0.02      0.63      0.41        87
   low_risk       1.00      0.57      0.70      0.73      0.63      0.40     17118

avg / total       0.99      0.57      0.70      0.73      0.63      0.40     17205

